<img src="http://oproject.org/tiki-download_file.php?fileId=6&display">
<img src="http://gfif.udea.edu.co/root/tmva/img/tmva_logo.gif">
# Required headers

In [ ]:
#include "TRInterface.h"
#include "TMVA/MethodC50.h"
#include "TMVA/MethodRSNNS.h"
#include "TMVA/MethodRXGB.h"

## Declare Factory

In [ ]:
TFile *inputFile  = TFile::Open( "inputdata.root"); 
TFile *outputFile = TFile::Open( "TMVAOutput.root", "RECREATE" );

TMVA::Tools::Instance();
TMVA::Factory *factory = new TMVA::Factory("TMVAClassification", outputFile,
                             "!V:ROC:!Correlations:!Silent:Color:!DrawProgressBar:AnalysisType=Classification" );

## Declare DataLoader

In [ ]:
TMVA::DataLoader *loader=new TMVA::DataLoader("dataset");

//adding variables to dataset
loader->AddVariable( "var1", 'F' );
loader->AddVariable( "var2", 'F' );
loader->AddVariable( "var3", 'F' );
loader->AddVariable( "var4", 'F' );

## Setting up Dataset

In [ ]:
TTree *tsignal     = (TTree*)inputFile->Get("Sig");
TTree *tbackground = (TTree*)inputFile->Get("Bkg");   
TCut mycuts = "";
TCut mycutb = "";
   
loader->AddSignalTree    ( tsignal,     1.0    );//signal weight     = 1
loader->AddBackgroundTree( tbackground, 1.0 );   //background weight = 1 
loader->PrepareTrainingAndTestTree( mycuts, mycutb,
"nTrain_Signal=1000:nTrain_Background=1000:SplitMode=Random:NormMode=NumEvents:!V" ); 


## Booking methods
Available Booking methods with options for RMVA are

C50 Boosted Decision Trees http://oproject.org/tiki-index.php?page=RMVA#C50Booking

RMLP Neural Networks http://oproject.org/tiki-index.php?page=RMVA#RSNNSMLP 

Extreme Gradient Boosted (RXGB) Decision Trees http://oproject.org/tiki-index.php?page=RMVA#RXGBBooking

In [ ]:
//C50 Boosted Decision Trees (BDTs)
factory->BookMethod(loader, TMVA::Types::kC50, "C50","!H:NTrials=5:Rules=kTRUE:ControlSubSet=kFALSE:ControlBands=10:ControlWinnow=kFALSE:ControlNoGlobalPruning=kTRUE:ControlCF=0.25:ControlMinCases=2:ControlFuzzyThreshold=kTRUE:ControlSample=0:ControlEarlyStopping=kTRUE:!V" );
   
//Neural Networks using RSNNS package
factory->BookMethod(loader, TMVA::Types::kRSNNS, "RMLP","!H:VarTransform=N:Size=c(5):Maxit=10:InitFunc=Randomize_Weights:LearnFunc=Std_Backpropagation:LearnFuncParams=c(0.2,0):!V" );

//eXtreme Gradient Boosted XGB Decision Trees
factory->BookMethod(loader, TMVA::Types::kRXGB, "RXGB","!V:NRounds=20:MaxDepth=2:Eta=1" );

//TMVA BDTs
factory->BookMethod(loader,TMVA::Types::kBDT, "BDT","!V:NTrees=50:MinNodeSize=2.5%:MaxDepth=2:BoostType=AdaBoost:AdaBoostBeta=0.5:UseBaggedBoost:BaggedSampleFraction=0.5:SeparationType=GiniIndex:nCuts=20" );

## Training the Methods

In [ ]:
factory->TrainAllMethods();

## Testing and Evaluating the data

In [ ]:
factory->TestAllMethods();
factory->EvaluateAllMethods();    

## Ploting ROC Curve

In [ ]:
%jsroot
TCanvas *c=factory->GetROCCurve(loader);
c->Draw();

## Cleaning up

In [ ]:
outputFile->Close();
delete factory;
delete loader;